<a href="https://colab.research.google.com/github/DrSchmerz/Telecom-Data/blob/main/Please_run_%E2%80%9EEoDA_2425_assignment1_ipynb%E2%80%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Engineering of Data Analysis: assignment 1

By delivering this notebook, we confirm that the code presented was developed by the following students.

**Student num:66286     ; Name:Hannes Reinhardt

**Student num:64737     ; Name: Philipp Oliver Laurence Goetting

**DEADLINE:** 21st April, 23h59

**Only one student should deliver the notebook**


**IMPORTANT:** To use GPUs in Colab, you need to go to the menu ```Edit > Notebook settings``` and select GPU as the hardware accelerator.

Some useful links:
* [ACM DEBS 2015 Grand Challenge](http://www.debs2015.org/call-grand-challenge.html)

* [Spark web site](https://spark.apache.org/)

* [Spark MLlib main page](https://spark.apache.org/mllib/)
* [Spark MLlib guide](https://spark.apache.org/docs/latest/ml-guide.html)

* [CuDF documentation](https://docs.rapids.ai/api/cudf/stable/)
* [cuML documentation](https://docs.rapids.ai/api/cuml/stable/)


# Install software

In [1]:
# Software to download files
!pip install gdown

In [2]:
# Install Spark
!apt-get install openjdk-17-jdk-headless
!pip install pyspark==4.0.0.dev2
!mkdir checkpoint


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-17-jdk-headless is already the newest version (17.0.14+7-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
mkdir: cannot create directory ‘checkpoint’: File exists


In [3]:
# Install RAPIDS - for using cuDF, cuML
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible,
# it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py


fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pynvml.py", line 2415, in _LoadNvmlLibrary
    nvmlLib = CDLL("libnvidia-ml.so.1")
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: libnvidia-ml.so.1: cannot open shared object file: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 18, in <module>
    pynvml.nvmlInit()
  File "/usr/local/lib/python3.11/dist-packages/pynvml.py", line 2387, in nvmlInit
    nvmlInitWithFlags(0)
  File "/usr/local/lib/python3.11/dist-packages/pynvml.py", line 2370, in nvmlInitWithFlags
    _LoadNvmlLibrary()
  File "/usr/local/lib/pyt

# Setup

The data sets are available in the following link: https://drive.google.com/drive/folders/1WMwLUj0t4Q0GSll96lbF2bDjaPVh1w8z?usp=sharing. For running in Google Colab, you should access the link and Add Shortcut to your Drive.



In [4]:
#!pip install pyspark pandas numpy matplotlib scikit-learn cudf cupy cuml

In [5]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import *
from pyspark.ml.feature import *
import pyspark.pandas as ps

import timeit
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.cluster

import cudf
import cupy as cp
import cuml.cluster
from google.colab import drive

/usr/local/lib/python3.11/dist-packages/pyspark/pandas/__init__.py:43: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 314, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path o

In [11]:
from google.colab import drive
import os

try:
    drive.mount('/content/drive')
except ValueError:
    print("Google Drive is already mounted at /content/drive")

# Large dataset - 12.3 GB compressed
#FILENAME = "/content/drive/MyDrive/assignment1/sorted_data.csv.gz"

# Small dataset - 135 MB compressed
#FILENAME = "/content/drive/MyDrive/assignment1/sample.csv.gz"

# Tiny dataset - 6.8 MB compressed (you can use this one for speeding up development only)
FILENAME = "/content/drive/MyDrive/assignment1/tiny.csv.gz"

# (Optional) Verify the file exists, to provide user feedback
if not os.path.exists(FILENAME):
    print(f"Error: Dataset file not found at {FILENAME}. ")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Exercise 1: simple statistics

Compute, for each license, the total amount of money collected.

Comapre the time it takes to execute the code for Pandas, Spark Pandas API, Spark SQL and cuDF.

**Draw some conclusions** by comparing the time for performing the computation using Spark and Pandas, and also when using the small and long dataset.

### Code: Pandas library

In [12]:
start_time = time.time()
mySchema = ["medallion", "hack_license", "pickup_datetime",
            "dropoff_datetime", "trip_time_in_secs", "trip_distance",
            "pickup_longitude", "pickup_latitude", "dropoff_longitude",
            "dropoff_latitude", "payment_type", "fare_amount",
            "surcharge", "mta_tax", "tip_amount",
            "tolls_amount", "total_amount"]

dataset = pd.read_csv(FILENAME,names=mySchema)
result = dataset[["hack_license","total_amount"]].groupby("hack_license").sum()

end_time = time.time()

print( "Runtime = " + str(end_time - start_time))


Runtime = 0.8054232597351074


### Results (Pandas)

The time to process the small dataset was : **was 8.62111210823059** seconds.

The time to process the large dataset was : **TO COMPLETE** seconds.

### Code: Spark Pandas API

In [13]:
start_time = time.time()
mySchema = ["medallion", "hack_license", "pickup_datetime",
            "dropoff_datetime", "trip_time_in_secs", "trip_distance",
            "pickup_longitude", "pickup_latitude", "dropoff_longitude",
            "dropoff_latitude", "payment_type", "fare_amount",
            "surcharge", "mta_tax", "tip_amount",
            "tolls_amount", "total_amount"]

dataset = ps.read_csv(FILENAME,names=mySchema)
result = dataset[["hack_license","total_amount"]].groupby("hack_license").sum()
# force execution
result0 = result.to_spark().collect()

end_time = time.time()

print( "Runtime = " + str(end_time - start_time))


/usr/local/lib/python3.11/dist-packages/pyspark/pandas/utils.py:1017: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/usr/local/lib/python3.11/dist-packages/pyspark/pandas/utils.py:1017: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/usr/local/lib/python3.11/dist-packages/pyspark/pandas/utils.py:1017: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


Runtime = 3.4201674461364746


### Results (Spark Pandas API)

The time to process the small dataset was : **20.158647775650024** seconds.

The time to process the large dataset was : **TO COMPLETE** seconds.

### Spark SQL

In [14]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Group project") \
    .getOrCreate()

start_time = time.time()
mySchema = StructType([
    StructField("medallion", StringType()),
    StructField("hack_license", StringType()),
    StructField("pickup_datetime", TimestampType()),
    StructField("dropoff_datetime", TimestampType()),
    StructField("trip_time_in_secs", IntegerType()),
    StructField("trip_distance", DoubleType()),
    StructField("pickup_longitude", DoubleType()),
    StructField("pickup_latitude", DoubleType()),
    StructField("dropoff_longitude", DoubleType()),
    StructField("dropoff_latitude", DoubleType()),
    StructField("payment_type", StringType()),
    StructField("fare_amount", DoubleType()),
    StructField("surcharge", DoubleType()),
    StructField("mta_tax", DoubleType()),
    StructField("tip_amount", DoubleType()),
    StructField("tolls_amount", DoubleType()),
    StructField("total_amount", DoubleType()),
])

dataset = spark.read.load(FILENAME, format="csv",
                         sep=",", schema=mySchema, header="false")
dataset.createOrReplaceTempView("data")
statisticsDF = spark.sql( """SELECT hack_license, SUM(total_amount) AS total_amount FROM data GROUP BY hack_license""")
statistics = statisticsDF.collect()

end_time = time.time()

print( "Runtime = " + str(end_time - start_time))


Runtime = 2.288220167160034


### Results (Spark)

The time to process the small dataset was : **6.135289669036865** seconds.

The time to process the large dataset was : **TO COMPLETE** seconds.

### Code: cuDF

In [17]:
####
#start_time = time.time()
#mySchema = ["medallion", "hack_license", "pickup_datetime",
#            "dropoff_datetime", "trip_time_in_secs", "trip_distance",
 #           "pickup_longitude", "pickup_latitude", "dropoff_longitude",
  #          "dropoff_latitude", "payment_type", "fare_amount",
   #         "surcharge", "mta_tax", "tip_amount",
    #        "tolls_amount", "total_amount"]

#dataset = cudf.read_csv(FILENAME,names=mySchema)
#result = dataset[["hack_license","total_amount"]].groupby("hack_license").sum()

#end_time = time.time()

#print( "Runtime = " + str(end_time - start_time))
####

MemoryError: std::bad_alloc: CUDA error at: /pyenv/versions/3.12.9/lib/python3.12/site-packages/librmm/include/rmm/mr/device/cuda_memory_resource.hpp:62: cudaErrorInsufficientDriver CUDA driver version is insufficient for CUDA runtime version

In [18]:
start_time = time.time()

mySchema = ["medallion", "hack_license", "pickup_datetime",
            "dropoff_datetime", "trip_time_in_secs", "trip_distance",
            "pickup_longitude", "pickup_latitude", "dropoff_longitude",
            "dropoff_latitude", "payment_type", "fare_amount",
            "surcharge", "mta_tax", "tip_amount",
            "tolls_amount", "total_amount"]

# Read the CSV file using pandas with the appropriate encoding if needed
dataset_pd = pd.read_csv(FILENAME, names=mySchema, delimiter=",", quotechar='"', encoding='utf-8')

# Convert the pandas DataFrame to a cuDF DataFrame
dataset = cudf.from_pandas(dataset_pd)

# Perform the groupby and sum operation
result = dataset[["hack_license","total_amount"]].groupby("hack_license").sum()

end_time = time.time()

print( "Runtime = " + str(end_time - start_time))

RuntimeError: Fatal CUDA error encountered at: /__w/cudf/cudf/cpp/src/bitmask/null_mask.cu:93: 35 cudaErrorInsufficientDriver CUDA driver version is insufficient for CUDA runtime version

In [ ]:
!nvidia-smi # for GPU information
!nvcc --version # for CUDA version

### Results (cuDF)

The time to process the small dataset was : **2.8160507678985596** seconds.

The time to process the large dataset was : **TO COMPLETE** seconds.

### Results discussion

**TO BE COMPLETED**

## Exercise 2

The code presented in the **Plotting heatmap** section includes the call to an registered Python UDF function. In this exercise we want to measure the impact of using the UDF function.

Run this exercise with the small dataset.

In [ ]:
start_time = time.time()
# Squares of 150 meters
latitudeStep = 0.0013474668
longitudeStep = 0.0017958
northLatitude = 40.95
southLatitude = northLatitude - 300 * latitudeStep
eastLongitude = -74.2
westLongitude = eastLongitude + 300 * longitudeStep

# function to round longitude to a point in the middle of the square
def longiRound( val):
    return ((val - eastLongitude) // longitudeStep) * longitudeStep + eastLongitude + longitudeStep / 2
spark.udf.register("longround", longiRound, DoubleType())

# function to round latitude to a point in the middle of the square
def latRound( l):
    return northLatitude - ((northLatitude - l) // latitudeStep) * latitudeStep - latitudeStep / 2
spark.udf.register("latround", latRound, DoubleType())

mySchema = StructType([
    StructField("medallion", StringType()),
    StructField("hack_license", StringType()),
    StructField("pickup_datetime", TimestampType()),
    StructField("dropoff_datetime", TimestampType()),
    StructField("trip_time_in_secs", IntegerType()),
    StructField("trip_distance", DoubleType()),
    StructField("pickup_longitude", DoubleType()),
    StructField("pickup_latitude", DoubleType()),
    StructField("dropoff_longitude", DoubleType()),
    StructField("dropoff_latitude", DoubleType()),
    StructField("payment_type", StringType()),
    StructField("fare_amount", DoubleType()),
    StructField("surcharge", DoubleType()),
    StructField("mta_tax", DoubleType()),
    StructField("tip_amount", DoubleType()),
    StructField("tolls_amount", DoubleType()),
    StructField("total_amount", DoubleType()),
])

dataset = spark.read.load(FILENAME, format="csv",
                         sep=",", schema=mySchema, header="false")

# Let's filter data outside of the box and build a grid
# Points in each square are mapped to the center of the square.
dataset.createOrReplaceTempView("data")
filteredDataDF = spark.sql( """SELECT medallion, hack_license, pickup_datetime,
                                    dropoff_datetime, trip_time_in_secs, trip_distance,
                                    longround(pickup_longitude) AS pickup_longitude,
                                    latround(pickup_latitude) AS pickup_latitude,
                                    longround(dropoff_longitude) AS dropoff_longitude,
                                    latround(dropoff_latitude) AS dropoff_latitude,
                                    payment_type, fare_amount, mta_tax,
                                    tip_amount, tolls_amount, total_amount
                                  FROM data
                                  WHERE pickup_longitude >= """ + str(eastLongitude) + """ AND
                                  pickup_longitude <=  """ + str(westLongitude) + """ AND
                                  dropoff_longitude >=  """ + str(eastLongitude) + """ AND
                                  dropoff_longitude <=  """ + str(westLongitude) + """ AND
                                  pickup_latitude <= """ + str(northLatitude) + """ AND
                                  pickup_latitude >= """ + str(southLatitude) + """ AND
                                  dropoff_latitude <=  """ + str(northLatitude) + """ AND
                                  dropoff_latitude >=  """ + str(southLatitude))
filteredDataDF.createOrReplaceTempView("data")

# Frequency for pickups
pickupsDF = spark.sql( """SELECT pickup_longitude, pickup_latitude, count(*) AS cnt
                                  FROM data
                                  GROUP BY pickup_longitude, pickup_latitude""")
pickups = pickupsDF.collect()

end_time = time.time()

print( "Runtime = " + str(end_time - start_time))


**Results (Spark SQL with UDF Python function)**

The time to process the small dataset was : **13.147183656692505** seconds.

Write the code in Spark SQL without using the auxiliary function.

In [ ]:

# THis part gonna stay the same

start_time = time.time()

# Squares of 150 meters
latitudeStep = 0.0013474668
longitudeStep = 0.0017958
northLatitude = 40.95
southLatitude = northLatitude - 300 * latitudeStep
eastLongitude = -74.2
westLongitude = eastLongitude + 300 * longitudeStep

mySchema = StructType([
    StructField("medallion", StringType()),
    StructField("hack_license", StringType()),
    StructField("pickup_datetime", TimestampType()),
    StructField("dropoff_datetime", TimestampType()),
    StructField("trip_time_in_secs", IntegerType()),
    StructField("trip_distance", DoubleType()),
    StructField("pickup_longitude", DoubleType()),
    StructField("pickup_latitude", DoubleType()),
    StructField("dropoff_longitude", DoubleType()),
    StructField("dropoff_latitude", DoubleType()),
    StructField("payment_type", StringType()),
    StructField("fare_amount", DoubleType()),
    StructField("surcharge", DoubleType()),
    StructField("mta_tax", DoubleType()),
    StructField("tip_amount", DoubleType()),
    StructField("tolls_amount", DoubleType()),
    StructField("total_amount", DoubleType()),
])

dataset = spark.read.load(FILENAME, format="csv",
                         sep=",", schema=mySchema, header="false")


# Intergrated Version instead of def long/lat we included into the select statement of our SQL
dataset.createOrReplaceTempView("data")
filteredDataDF = spark.sql(f"""
SELECT
    medallion, hack_license, pickup_datetime,
    dropoff_datetime, trip_time_in_secs, trip_distance,

    -- Round pickup_longitude
    (FLOOR((pickup_longitude - {eastLongitude}) / {longitudeStep}) * {longitudeStep} +
     {eastLongitude} + {longitudeStep} / 2) AS pickup_longitude,

    -- Round pickup_latitude
    ({northLatitude} - FLOOR(({northLatitude} - pickup_latitude) / {latitudeStep}) * {latitudeStep} -
     {latitudeStep} / 2) AS pickup_latitude,

    -- Round dropoff_longitude
    (FLOOR((dropoff_longitude - {eastLongitude}) / {longitudeStep}) * {longitudeStep} +
     {eastLongitude} + {longitudeStep} / 2) AS dropoff_longitude,

    -- Round dropoff_latitude
    ({northLatitude} - FLOOR(({northLatitude} - dropoff_latitude) / {latitudeStep}) * {latitudeStep} -
     {latitudeStep} / 2) AS dropoff_latitude,

    payment_type, fare_amount, mta_tax, tip_amount, tolls_amount, total_amount
FROM data
WHERE pickup_longitude BETWEEN {eastLongitude} AND {westLongitude}
  AND dropoff_longitude BETWEEN {eastLongitude} AND {westLongitude}
  AND pickup_latitude BETWEEN {southLatitude} AND {northLatitude}
  AND dropoff_latitude BETWEEN {southLatitude} AND {northLatitude}
""")



filteredDataDF.createOrReplaceTempView("data")

# Frequency for pickups
pickupsDF = spark.sql( """SELECT pickup_longitude, pickup_latitude, count(*) AS cnt
                                  FROM data
                                  GROUP BY pickup_longitude, pickup_latitude""")
pickups = pickupsDF.collect()

end_time = time.time()

print( "Runtime = " + str(end_time - start_time))



**Results (Spark SQL with no UDF Python function)**

The time to process the small dataset was : **9.010905504226685** seconds.




### Discussion of results

Replacing Python UDFs with native SQL expressions results in better performance, more efficient resource usage, and faster query execution — especially when working with large-scale geospatial data like taxi trip records.

*UDFs recommended:*

•⁠  ⁠When the logic is too complex to be expressed directly in SQL.

•⁠  ⁠When prioritizing code readability, modularity, or reusability, even if it comes with some performance trade-offs.

•⁠  ⁠When working with data types or formats that are not natively supported by Spark SQL.

## Exercise 3

Let's start by trying to help the city to identify which new express bus routes shoud introduce. To this end, you should find the 20 most frequent routes whose distance is above a given treshold (defined by you).

For establishing these routes, we suggest that you use a grid of 500m of side.

Write two solutions: one using Spark SQL and the other Spark Pandas API.

In [ ]:
# COMPLETE with Spark SQL code


In [ ]:
# Squares of ~500 meters

latitudeStep = 0.004491556
longitudeStep = 0.005986
northLatitude = 40.95
southLatitude = northLatitude - 300 * latitudeStep
eastLongitude = -74.2
westLongitude = eastLongitude + 300 * longitudeStep

In [ ]:
routesDF = spark.sql("""
SELECT
    pickup_longitude,
    pickup_latitude,
    dropoff_longitude,
    dropoff_latitude,
    COUNT(*) AS cnt,
    AVG(trip_distance) AS avg_trip_distance_miles
FROM data
GROUP BY
    pickup_longitude,
    pickup_latitude,
    dropoff_longitude,
    dropoff_latitude
ORDER BY cnt DESC
LIMIT 100
""")
routesDF.show(20)

In [ ]:
# Average Longest Trip Distances

routesDF = spark.sql("""
SELECT
    pickup_longitude,
    pickup_latitude,
    dropoff_longitude,
    dropoff_latitude,
    COUNT(*) AS cnt,
    AVG(trip_distance) AS avg_trip_distance_miles
FROM data
GROUP BY
    pickup_longitude,
    pickup_latitude,
    dropoff_longitude,
    dropoff_latitude
ORDER BY avg_trip_distance_miles DESC, cnt DESC
LIMIT 100
""")
routesDF.show(20)

In [ ]:
# Sorting via Distandce and count
routesDF.orderBy(["avg_trip_distance_miles", "cnt"], ascending=[False, False]).show(50)

In [ ]:
# Defininng a treshhold based on avg trip distance 10 miles or bigger

# Start the timer
start_time = time.time()

routesDF = spark.sql("""
SELECT
    pickup_longitude,
    pickup_latitude,
    dropoff_longitude,
    dropoff_latitude,
    COUNT(*) AS cnt,
    AVG(trip_distance) AS avg_trip_distance_miles
FROM data
GROUP BY
    pickup_longitude,
    pickup_latitude,
    dropoff_longitude,
    dropoff_latitude
HAVING avg_trip_distance_miles >= 15
ORDER BY cnt DESC
LIMIT 100
""")

# Show the top results
routesDF.show(20)

# Stop the timer after query execution
end_time = time.time()

# Calculate the runtime
runtime = end_time - start_time
print(f"SQL query execution time: {runtime} seconds")

In [ ]:
start_time = time.time()

routesDF = spark.sql("""
SELECT
    pickup_longitude,
    pickup_latitude,
    dropoff_longitude,
    dropoff_latitude,
    COUNT(*) AS cnt,
    AVG(trip_distance) AS avg_trip_distance_miles
FROM data
GROUP BY
    pickup_longitude,
    pickup_latitude,
    dropoff_longitude,
    dropoff_latitude
HAVING avg_trip_distance_miles >= 15
ORDER BY cnt DESC
LIMIT 100
""")

# Stop the timer after query execution
end_time = time.time()

# Calculate the runtime
runtime = end_time - start_time
print(f"SQL query execution time: {runtime} seconds")

In [ ]:
# COMPLETE with Spark Pandas API code

# Start performance timer
start_time = time.time()

# Load the dataset using Spark's Pandas API (assumes CSV format with header)
psdf = ps.read_csv(FILENAME, header=None, names=[
    "medallion", "hack_license", "pickup_datetime", "dropoff_datetime",
    "trip_time_in_secs", "trip_distance", "pickup_longitude", "pickup_latitude",
    "dropoff_longitude", "dropoff_latitude", "payment_type", "fare_amount",
    "surcharge", "mta_tax", "tip_amount", "tolls_amount", "total_amount"
])

# Filter rows where trip distance > 15 miles
filtered_psdf = psdf[psdf["trip_distance"] > 15]

# Group by pickup and dropoff coordinates
grouped = filtered_psdf.groupby([
    "pickup_longitude", "pickup_latitude",
    "dropoff_longitude", "dropoff_latitude"
])

# Aggregate to get count and average trip distance
routes_summary = grouped.agg(
    cnt=("trip_distance", "count"),
    avg_trip_distance_miles=("trip_distance", "mean")
).reset_index()

# Sort by average distance and count (both descending)
sorted_routes = routes_summary.sort_values(
    by=["avg_trip_distance_miles", "cnt"],
    ascending=[False, False]
)

# Show top 20 routes
print(sorted_routes.head(20))

# Stop performance timer
end_time = time.time()
print(f"Execution time (Spark Pandas API): {end_time - start_time:.2f} seconds")

#### Discussion

**COMPLETE: Explain your code and discuss which one is preferable from your point of view**

We adjusted the squares as suggested in the exercise and then analyzed the most frequent routes, incorporating the average trip distance. With the small dataset, most trip distances were found to be between 1 and 3 miles. We then set a threshold of 15 miles and above for the average trip distances and recalculated the top 20 routes.

When comparing Spark SQL and Spark Pandas API, using the small dataset, we observed that SQL executed significantly faster than Spark Pandas API. While, we are more familiar with Pandas we would still choose SQL for better performance.

## Exercise 4

This question intends to define the location of taxi ranks / taxi stand (the places where taxis stop waiting for collecting clients) in a way that tries to minimize the distance a client needs to walk to reach a taxi rank.

Run this exercise with the small dataset and compare the execution time when using scikit-learn, cuML and Spark MLib.

**Note:** This dataset is for NYC taxis. So, pickups outside of the city are infrequent and not representative of the demand in such areas. As such, you should focus on pickups in a square that includes NYC (it is ok if the square includes parts outside of the city). Use, for example, the following square:
```
northLatitude = 40.86
southLatitude = 40.68
eastLongitude = -74.03
westLongitude = -73.92
```

**Suggestion:** Plot your results as a heatmap, with the color being a measurement of the value of the taxi rank.

To find important taxi stands in NYC we using a clustering analysis.
- Map of NYC with the square (in red)
- Elbow (based on the tiny dataset)
- Cluster Analysis
- Performance Comparison

----

In [ ]:
pip install geopandas shapely contextily matplotlib

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, box
import matplotlib.pyplot as plt
import contextily as ctx

# Step 1: Define your target box in EPSG:4326 (lat/lon)
northLatitude = 40.86
southLatitude = 40.68
eastLongitude = -74.03
westLongitude = -73.92

highlight_box = box(eastLongitude, southLatitude, westLongitude, northLatitude)
highlight_gdf = gpd.GeoDataFrame({'geometry': [highlight_box]}, crs="EPSG:4326").to_crs(epsg=3857)

# Convert pickup data to GeoDataFrame, including 'cnt' column
pickup_points = gpd.GeoDataFrame(
    pickups,
    geometry=[Point(row.pickup_longitude, row.pickup_latitude) for row in pickups],
    crs="EPSG:4326"  # WGS84
).to_crs(epsg=3857)  # Project to match NYC map/grid

pickup_points['cnt'] = [row.cnt for row in pickups]

# Same for dropoffs
dropoff_points = gpd.GeoDataFrame(
    dropoffs,
    geometry=[Point(row.dropoff_longitude, row.dropoff_latitude) for row in dropoffs],
    crs="EPSG:4326"
).to_crs(epsg=3857)

dropoff_points['cnt'] = [row.cnt for row in dropoffs]

# Step 2: Plot full map + grid + highlighted box
fig, ax = plt.subplots(figsize=(12, 12))

# Plot NYC boundary and grid
nyc.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1)
grid.plot(ax=ax, facecolor='none', edgecolor='gray', linewidth=0.5)

# Step 3: Plot pickups with intensity based on frequency
pickup_points.plot(
    ax=ax,
    column='cnt',  # Color by count
    cmap='rainbow',
    markersize=5,
    alpha=0.6,
    legend=True
)

# Step 4: Optionally, plot dropoffs similarly (if needed)
# dropoff_points.plot(ax=ax, column='cnt', cmap='cool', markersize=2, alpha=0.5)

# Step 5: Highlight the defined box region
highlight_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=3, linestyle='--')

# Add basemap and styling
ctx.add_basemap(ax, crs=nyc.crs, source=ctx.providers.CartoDB.Positron)

# Title
ax.set_title("NYC Pickup and Dropoff Density with Highlighted Region")

# Turn off axis for better map presentation
plt.axis('off')
plt.show()


In [ ]:
----

In [ ]:
# ELbow Technique to determine number of clusters only execute on small df
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

inertias_sklearn = []
K_range = range(1, 11)

for k in K_range:
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(df)
    inertias_sklearn.append(model.inertia_)

# Plot
plt.figure(figsize=(8, 5))
plt.plot(K_range, inertias_sklearn, 'o-', label='scikit-learn')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia (Within-cluster SSE)')
plt.title('Elbow Method for Optimal k (scikit-learn)')
plt.grid(True)
plt.show()

**Optimal Cluster number 3**

In [ ]:
# Filter for NYC bounds
northLatitude = 40.86
southLatitude = 40.68
eastLongitude = -74.03
westLongitude = -73.92

df_filtered = df[
    (df['pickup_latitude'] >= southLatitude) & (df['pickup_latitude'] <= northLatitude) &
    (df['pickup_longitude'] >= westLongitude) & (df['pickup_longitude'] <= eastLongitude)
][['pickup_longitude', 'pickup_latitude']]


In [ ]:
# COMPLETE with code using scikit-learn

In [ ]:
from sklearn.cluster import KMeans
import time

# Start timer
start = time.time()

# Fit KMeans to pickup points within the NYC bounds and grid
kmeans_sklearn = KMeans(n_clusters=3, random_state=42)
kmeans_sklearn.fit(df_filtered)  # Fit to the filtered DataFrame

# End timer
end = time.time()
print(f"scikit-learn KMeans execution time: {end - start:.2f} seconds")

# Get centroids
centroids_sklearn = kmeans_sklearn.cluster_centers_

In [ ]:
# COMPLETE with code using cuML


In [ ]:
from cuml.cluster import KMeans as cuKMeans
import cupy as cp
import time
import cudf

# 1. Data Loading with Timing
start_load = time.time()
df = cudf.read_csv(FILENAME)  # Direct cuDF loading
end_load = time.time()
print(f"Data loading time (cuDF): {end_load - start_load:.2f} seconds")

# 2. Define NYC bounding box
northLatitude = 40.86
southLatitude = 40.68
eastLongitude = -74.03
westLongitude = -73.92

# 3. GPU-Based Filtering with Timing
start_filter = time.time()
df_filtered = df[
    (df['pickup_latitude'] >= southLatitude) & (df['pickup_latitude'] <= northLatitude) &
    (df['pickup_longitude'] >= westLongitude) & (df['pickup_longitude'] <= eastLongitude)
]
end_filter = time.time()
print(f"Filtering time (cuDF): {end_filter - start_filter:.2f} seconds")

# 4. Convert to cupy array
X_cuml = df_filtered[['pickup_longitude', 'pickup_latitude']].values

# 5. KMeans Clustering with Timing
start_kmeans = time.time()
kmeans_cuml = cuKMeans(n_clusters=3, random_state=42, init='k-means++', max_iter=300, tol=1e-4)
kmeans_cuml.fit(X_cuml)
end_kmeans = time.time()
print(f"cuML KMeans execution time: {end_kmeans - start_kmeans:.2f} seconds")

# 6. Get centroids
centroids_cuml = kmeans_cuml.cluster_centers_

# Total execution time (excluding data loading)
total_time = (end_filter - start_filter) + (end_kmeans - start_kmeans)
print(f"Total execution time (filtering + KMeans): {total_time:.2f} seconds")

In [ ]:
# COMPLETE with code using Spark Mlib

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans as SparkKMeans
import time

# Start Spark session
spark = SparkSession.builder.appName("TaxiKMeans").getOrCreate()

# Create Spark DataFrame
spark_df = spark.createDataFrame(df)

# Assemble features
vec_assembler = VectorAssembler(
    inputCols=["pickup_longitude", "pickup_latitude"],
    outputCol="features"
)
spark_vec_df = vec_assembler.transform(spark_df)

# Run KMeans
start = time.time()
kmeans = SparkKMeans(k=3, seed=42)
model = kmeans.fit(spark_vec_df)
end = time.time()

print(f"Spark MLlib KMeans execution time: {end - start:.2f} seconds")

# Get centroids
centroids_spark = model.clusterCenters()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans as SparkKMeans
import time

# ... (Previous code for grid filtering) ...


# Start Spark session (if not already started)
# spark = SparkSession.builder.appName("TaxiKMeans").getOrCreate()

# Create Spark DataFrame from filtered data
spark_df_filtered = spark.createDataFrame(df_filtered)  # Use df_filtered

# Assemble features
vec_assembler = VectorAssembler(
    inputCols=["pickup_longitude", "pickup_latitude"],
    outputCol="features"
)
spark_vec_df = vec_assembler.transform(spark_df_filtered)  # Use filtered data

# Run KMeans
start = time.time()
kmeans = SparkKMeans(k=3, seed=42)  # k=3 for consistency
model = kmeans.fit(spark_vec_df)
end = time.time()

print(f"Spark MLlib KMeans execution time: {end - start:.2f} seconds")

# Get centroids
centroids_spark = model.clusterCenters()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans as SparkKMeans
import time

# ... (Previous code to define grid, UDFs, etc.) ...

# Define NYC bounding box
northLatitude = 40.86
southLatitude = 40.68
eastLongitude = -74.03
westLongitude = -73.92

# Start Spark session (if not already started)
# spark = SparkSession.builder.appName("TaxiKMeans").getOrCreate()

# Filter the Spark DataFrame directly
spark_df_filtered = spark_df.filter(
    (col("pickup_latitude") >= southLatitude) & (col("pickup_latitude") <= northLatitude) &
    (col("pickup_longitude") >= westLongitude) & (col("pickup_longitude") <= eastLongitude)
)

# Assemble features
vec_assembler = VectorAssembler(
    inputCols=["pickup_longitude", "pickup_latitude"],
    outputCol="features"
)
spark_vec_df = vec_assembler.transform(spark_df_filtered)

# Run KMeans
start = time.time()
kmeans = SparkKMeans(k=3, seed=42)  # k=3 for consistency
model = kmeans.fit(spark_vec_df)
end = time.time()

print(f"Spark MLlib KMeans execution time: {end - start:.2f} seconds")

# Get centroids
centroids_spark = model.clusterCenters()

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, box
import matplotlib.pyplot as plt
import contextily as ctx
from sklearn.cluster import KMeans

# Step 1: Define your target box in EPSG:4326 (lat/lon)
northLatitude = 40.86
southLatitude = 40.68
eastLongitude = -74.03
westLongitude = -73.92

highlight_box = box(eastLongitude, southLatitude, westLongitude, northLatitude)
highlight_gdf = gpd.GeoDataFrame({'geometry': [highlight_box]}, crs="EPSG:4326").to_crs(epsg=3857)

# Step 2: Check the column names to ensure we reference them correctly
print(pickups.columns)  # Check column names in pickups DataFrame
print(dropoffs.columns)  # Check column names in dropoffs DataFrame

# Step 3: Convert pickup data to GeoDataFrame, including 'cnt' column
# Replace 'pickup_longitude' and 'pickup_latitude' with the correct column names.
pickup_points = gpd.GeoDataFrame(
    pickups,
    geometry=[Point(row['pickup_longitude'], row['pickup_latitude']) for idx, row in pickups.iterrows()],
    crs="EPSG:4326"  # WGS84
).to_crs(epsg=3857)  # Project to match NYC map/grid

pickup_points['cnt'] = pickups['cnt']  # Assuming you have a 'cnt' column in pickups DataFrame

# Same for dropoffs
dropoff_points = gpd.GeoDataFrame(
    dropoffs,
    geometry=[Point(row['dropoff_longitude'], row['dropoff_latitude']) for idx, row in dropoffs.iterrows()],
    crs="EPSG:4326"
).to_crs(epsg=3857)

dropoff_points['cnt'] = dropoffs['cnt']  # Assuming you have a 'cnt' column in dropoffs DataFrame

# Step 4: Run KMeans clustering on pickups (correctly referencing the columns)
# You may need to replace 'pickup_longitude' and 'pickup_latitude' with the actual column names from your data.
kmeans = KMeans(n_clusters=3, random_state=42)
pickup_points['cluster'] = kmeans.fit_predict(pickup_points[['pickup_longitude', 'pickup_latitude']])

# Step 5: Plot full map + grid + highlighted box
fig, ax = plt.subplots(figsize=(12, 12))

# Plot NYC boundary and grid
nyc.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1)
grid.plot(ax=ax, facecolor='none', edgecolor='gray', linewidth=0.5)

# Step 6: Plot pickups with intensity based on frequency
pickup_points.plot(
    ax=ax,
    column='cnt',  # Color by count
    cmap='rainbow',
    markersize=5,
    alpha=0.6,
    legend=True
)

# Step 7: Plot clusters with different colors
cluster_colors = {0: 'red', 1: 'blue', 2: 'green'}
for cluster in pickup_points['cluster'].unique():
    cluster_points = pickup_points[pickup_points['cluster'] == cluster]
    cluster_points.plot(
        ax=ax,
        color=cluster_colors[cluster],
        markersize=10,
        label=f"Cluster {cluster+1}"
    )

# Step 8: Plot dropoffs if needed (optional)
dropoff_points.plot(ax=ax, column='cnt', cmap='cool', markersize=2, alpha=0.5, legend=True)

# Step 9: Highlight the defined box region
highlight_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=3, linestyle='--')

# Add basemap and styling
ctx.add_basemap(ax, crs=nyc.crs, source=ctx.providers.CartoDB.Positron)

# Title
ax.set_title("NYC Pickup and Dropoff Density with Clusters and Highlighted Region")

# Turn off axis for better map presentation
plt.axis('off')
plt.legend()
plt.show()


----

In [ ]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np
!pip install geodatasets
from geodatasets import get_path

# Step 1: Load NYC boundary using geodatasets
# NYC borough boundaries from a common public dataset
nyc = gpd.read_file(get_path("nybb")).to_crs(epsg=3857)  # Web Mercator

# Step 2: Create a 500-meter grid over NYC
def create_grid(gdf, grid_size):
    bounds = gdf.total_bounds  # (minx, miny, maxx, maxy)
    xmin, ymin, xmax, ymax = bounds
    rows = int(np.ceil((ymax - ymin) / grid_size))
    cols = int(np.ceil((xmax - xmin) / grid_size))

    grid_cells = []
    for i in range(cols):
        for j in range(rows):
            x0 = xmin + i * grid_size
            x1 = x0 + grid_size
            y0 = ymin + j * grid_size
            y1 = y0 + grid_size
            grid_cells.append(box(x0, y0, x1, y1))

    grid = gpd.GeoDataFrame({'geometry': grid_cells}, crs=gdf.crs)
    return grid

grid_size_m = 500  # 500 meters
grid = create_grid(nyc, grid_size_m)

# Optional: Clip grid to NYC shape
grid = gpd.overlay(grid, nyc, how="intersection")

# Step 3: Plot everything
fig, ax = plt.subplots(figsize=(12, 12))
nyc.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1)
grid.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=0.5)
ctx.add_basemap(ax, crs=nyc.crs, source=ctx.providers.CartoDB.Positron)
ax.set_title("NYC with 500m Grid")
plt.axis('off')
plt.show()

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Convert pickup data to GeoDataFrame, including 'cnt' column
pickup_points = gpd.GeoDataFrame(
    pickups,
    geometry=[Point(row.pickup_longitude, row.pickup_latitude) for row in pickups],
    crs="EPSG:4326"  # WGS84
).to_crs(epsg=3857)  # Project to match NYC map/grid

# Explicitly add the 'cnt' column to the GeoDataFrame
pickup_points['cnt'] = [row.cnt for row in pickups]


# (Optional) Same for dropoffs
dropoff_points = gpd.GeoDataFrame(
    dropoffs,
    geometry=[Point(row.dropoff_longitude, row.dropoff_latitude) for row in dropoffs],
    crs="EPSG:4326"
).to_crs(epsg=3857)

# Add a 'cnt' column for dropoffs as well (if needed)
dropoff_points['cnt'] = [row.cnt for row in dropoffs]

# Plot on top of NYC map + grid
fig, ax = plt.subplots(figsize=(12, 12))
nyc.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1)
grid.plot(ax=ax, facecolor='none', edgecolor='gray', linewidth=0.5)

# Plot pickups with intensity based on frequency
pickup_points.plot(
    ax=ax,
    column='cnt',  # Color by count
    cmap='rainbow',
    markersize=5,
    alpha=0.6,
    legend=True
)

# Optional: overlay dropoffs similarly
# dropoff_points.plot(ax=ax, column='cnt', cmap='cool', markersize=2, alpha=0.5)

ctx.add_basemap(ax, crs=nyc.crs, source=ctx.providers.CartoDB.Positron)
ax.set_title("NYC Pickup Density with 500m Grid")
plt.axis('off')
plt.show()

In [ ]:
from shapely.geometry import box
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

# Step 1: Define your target box in EPSG:4326 (lat/lon)
northLatitude = 40.86
southLatitude = 40.68
eastLongitude = -74.03
westLongitude = -73.92

highlight_box = box(eastLongitude, southLatitude, westLongitude, northLatitude)
highlight_gdf = gpd.GeoDataFrame({'geometry': [highlight_box]}, crs="EPSG:4326").to_crs(epsg=3857)

# Step 2: Plot full map + grid + highlighted box
fig, ax = plt.subplots(figsize=(12, 12))

# Full NYC boundary and grid
nyc.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1)
grid.plot(ax=ax, facecolor='none', edgecolor='gray', linewidth=0.5)

# (Optional) pickup points if available
# pickup_points.plot(ax=ax, column='cnt', cmap='rainbow', markersize=2, alpha=0.5)

# Step 3: Highlighted box in a bright color
highlight_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=3, linestyle='--')

# Add basemap and styling
ctx.add_basemap(ax, crs=nyc.crs, source=ctx.providers.CartoDB.Positron)
ax.set_title("NYC Map with Highlighted Region")
plt.axis('off')
plt.show()


In [ ]:
# Load only the pickup longitude and latitude
df = pd.read_csv(FILENAME, usecols=[6, 7], names=[
    "pickup_longitude", "pickup_latitude"], skiprows=1)

# Define NYC bounding box
northLatitude = 40.86
southLatitude = 40.68
eastLongitude = -74.03
westLongitude = -73.92

# Filter the data within bounding box
df = df.dropna()
df = df[
    (df["pickup_latitude"].between(southLatitude, northLatitude)) &
    (df["pickup_longitude"].between(eastLongitude, westLongitude))
]

# Sample after filtering (IMPORTANT for performance)
#sample_df = df.sample(n=5000, random_state=42)

# Extract values for clustering
X = df[["pickup_longitude", "pickup_latitude"]].values

In [ ]:
# COMPLETE with code using scikit-learn
# COMPLETE with code using scikit-learn
import seaborn as sns
from sklearn.cluster import KMeans


# KMeans Clustering
start = time.time()
kmeans = KMeans(n_clusters=3, random_state=42).fit(X)
end = time.time()

print(f"Scikit-learn KMeans Time: {end - start:.2f}s")

# Visualize heatmap (as density or cluster centers)
centroids = kmeans.cluster_centers_

plt.figure(figsize=(8, 6))
sns.kdeplot(x=X[:, 0], y=X[:, 1], fill=True, cmap="Reds", alpha=0.6, bw_adjust=0.5)
plt.scatter(centroids[:, 0], centroids[:, 1], c='blue', marker='X', s=100, label='Taxi Ranks')
plt.title("Taxi Rank Heatmap (Scikit-learn)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.show()

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx
import matplotlib.pyplot as plt
from geodatasets import get_path

# ── Your square
northLatitude = 40.86
southLatitude = 40.68
eastLongitude = -74.03
westLongitude = -73.92
highlight_box = box(eastLongitude, southLatitude, westLongitude, northLatitude)
highlight_gdf = (
    gpd.GeoDataFrame({'geometry': [highlight_box]}, crs="EPSG:4326")
      .to_crs(epsg=3857)
)

# ── Load and project NYC
nyc = gpd.read_file(get_path("nybb")).to_crs(epsg=3857)

# ── Convert centroids to GeoDataFrame
#    `centroids` is your array of shape (n_clusters, 2) in [lon, lat]
centroid_pts = gpd.GeoDataFrame(
    geometry=[Point(lon, lat) for lon, lat in centroids],
    crs="EPSG:4326"
).to_crs(epsg=3857)

# ── Plot everything
fig, ax = plt.subplots(figsize=(12, 12))
nyc.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1)
highlight_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=3, linestyle='--')

# Plot centroids
centroid_pts.plot(
    ax=ax,
    marker='X',
    color='blue',
    markersize=150,
    label='Cluster Centers'
)

ctx.add_basemap(ax, crs=nyc.crs, source=ctx.providers.CartoDB.Positron)

ax.set_title("NYC with Taxi Rank Clusters & Highlighted Area")
ax.legend()
ax.set_axis_off()
plt.show()

In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
import contextily as ctx

# ── 1) Define bbox and load/project NYC just once
northLatitude = 40.86
southLatitude = 40.68
eastLongitude = -74.03
westLongitude = -73.92

# make box GeoDataFrame
bbox = box(eastLongitude, southLatitude, westLongitude, northLatitude)
bbox_gdf = gpd.GeoDataFrame({'geometry':[bbox]}, crs="EPSG:4326").to_crs(epsg=3857)

# your highlight box (same as bbox here)
highlight_gdf = bbox_gdf.copy()

# load NYC once and project
nyc = gpd.read_file(get_path("nybb")).to_crs(epsg=3857)

# ── 2) Project all pickups to Web Mercator
pickup_merc = pickup_points.to_crs(epsg=3857)

# ── 3) Clip to the area of interest
pickup_clip = gpd.clip(pickup_merc, bbox_gdf)
nyc_clip    = gpd.clip(nyc,    bbox_gdf)

# ── 4) Extract X/Y coords for density
xs = pickup_clip.geometry.x.values
ys = pickup_clip.geometry.y.values

# choose number of bins for resolution
nx, ny = 200, 200
heatmap, xedges, yedges = np.histogram2d(xs, ys, bins=[nx, ny])

# compute extent in map coords
extent = [xedges.min(), xedges.max(), yedges.min(), yedges.max()]

# ── 5) Plot clipped map + heatmap + box + centroids
fig, ax = plt.subplots(figsize=(10, 10))

# basemap + clipped street outlines
nyc_clip.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.5)

# density heatmap
ax.imshow(
    heatmap.T,             # transpose to align x/y
    extent=extent,         # put it in the right place
    origin='lower',        # y-axis goes bottom→top
    cmap='Reds',           # your heatmap palette
    alpha=0.6
)

# highlight box border
highlight_gdf.plot(
    ax=ax,
    facecolor='none',
    edgecolor='blue',
    linewidth=2,
    linestyle='--',
    label='Focus Area'
)

# optional: plot your KMeans centers
centroid_pts.to_crs(epsg=3857).plot(
    ax=ax,
    marker='X',
    color='white',
    edgecolor='black',
    markersize=100,
    label='Cluster Centers'
)

# add basemap tiles under everything
ctx.add_basemap(ax, crs=nyc.crs, source=ctx.providers.CartoDB.Positron)

ax.set_title("Taxi Pickup Density & Cluster Centers\nZoomed to 40.68–40.86 N, 74.03–73.92 W")
ax.legend()
ax.set_axis_off()
plt.tight_layout()
plt.show()

In [ ]:
# COMPLETE with code using cuML


In [ ]:

# COMPLETE with code using Spark Mlib

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans as SparkKMeans
import time

# Start Spark
spark = SparkSession.builder.appName("TaxiKMeans").getOrCreate()

# Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Assemble features
vec_assembler = VectorAssembler(
    inputCols=["pickup_longitude", "pickup_latitude"],
    outputCol="features"
)
spark_vec_df = vec_assembler.transform(spark_df)

# Run KMeans
start = time.time()
kmeans = SparkKMeans(k=3, seed=42)
model = kmeans.fit(spark_vec_df)
end = time.time()

print(f"Spark MLlib KMeans execution time: {end - start:.2f} seconds")

# Get cluster centers
centroids_spark = model.clusterCenters()


In [ ]:
# Convert Spark centroids to numpy array
centroids_np = np.array(centroids_spark)

# Plot heatmap with seaborn
plt.figure(figsize=(10, 8))

# Heatmap of pickup density
sns.kdeplot(
    x=df["pickup_longitude"],
    y=df["pickup_latitude"],
    fill=True,
    cmap="Reds",
    alpha=0.6,
    bw_adjust=0.5
)

# Overlay the Spark KMeans centroids
plt.scatter(
    centroids_np[:, 0],  # longitudes
    centroids_np[:, 1],  # latitudes
    color='blue',
    marker='X',
    s=150,
    label='Suggested Taxi Ranks (Spark KMeans)'
)

plt.title("Taxi Pickup Heatmap with Spark MLlib Cluster Centers")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.grid(True)
plt.show()

**Execution times:**

scikit-learn: **TO COMPLETE**

cuML: **TO COMPLETE**

Spark Mlib: **TO COMPLETE**


#### Discussion

**COMPLETE: Explain the rationale of your solution, your code and discuss results**

scikit-learn is optimized for small to medium-sized datasets and runs efficiently on a single machine. It's fast, easy to use, and integrates well with Python's data science ecosystem, making it ideal for local development, rapid prototyping, and small-scale tasks. On the other hand, Spark MLlib is built for large-scale data processing in distributed environments, making it suitable for big data workloads that can't fit into a single machine's memory. It leverages the power of cluster computing to handle massive datasets and is integrated into the larger Spark ecosystem, which is commonly used in production-level data pipelines. While scikit-learn excels in simplicity and speed for smaller tasks, Spark MLlib shines when dealing with large-scale data and distributed computing needs.